In [1]:
import os
import sys

__FILE_DIR_PATH = os.path.dirname(__vsc_ipynb_file__) \
    if __IPYTHON__ \
    else os.path.dirname(__file__)

WORKSPACE_HOME = __FILE_DIR_PATH.replace(os.path.basename(__FILE_DIR_PATH), "")
WORKSPACE_HOME = WORKSPACE_HOME.replace("/trunk", "")
DATASET_DIR_HOME = f"{WORKSPACE_HOME}/data/colley"

print(WORKSPACE_HOME)
sys.path.append(WORKSPACE_HOME)

from pandas import DataFrame

from core import *
from ipirec import *
from colley import *
from decompositions import *

/home/taegyu/git_repo/ipirec/


/home/taegyu/anaconda3/envs/cfEnv/lib/python3.11/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


In [2]:
_FOLD_SET_ID = 4
top_n_conditions = [n for n in range(3, 37, 2)]

_TEST_SET_FILES_LIST = [
    str.format(
        "{0}/test_{1}_{2}_list.csv",
        DATASET_DIR_HOME,
        _FOLD_SET_ID,
        DecisionType.to_str(d),
    )
    for d in [
        DecisionType.E_LIKE,
        DecisionType.E_PURCHASE,
    ]
]

dataset = ColleyFilteredDataSet(dataset_dir_path=DATASET_DIR_HOME)
dataset._load_metadata_()
for decision_type in DecisionType:
    dataset.append_decisions(
        file_path=str.format(
            "{0}/train_{1}_{2}_list.csv",
            DATASET_DIR_HOME,
            _FOLD_SET_ID,
            DecisionType.to_str(decision_type),
        ),
        decision_type=decision_type,
    )
dataset.__id_index_mapping__()

[IO] /home/taegyu/git_repo/ipirec//data/colley/train_4_view_list.csv


[LOAD] train_4_view_list.csv: 100%|██████████| 46568/46568 [00:00<00:00, 81709.91it/s]


[IO] /home/taegyu/git_repo/ipirec//data/colley/train_4_like_list.csv


[LOAD] train_4_like_list.csv: 100%|██████████| 22362/22362 [00:00<00:00, 82316.18it/s]


[IO] /home/taegyu/git_repo/ipirec//data/colley/train_4_purchase_list.csv


[LOAD] train_4_purchase_list.csv: 100%|██████████| 12349/12349 [00:00<00:00, 82577.83it/s]


In [3]:
model_params = NMFDecompositionModel.create_models_parameters(
    factors_dim=200,
    factorizer_iters=200,
)
model = NMFDecompositionModel(dataset=dataset, model_params=model_params)
model.analysis()

[DecompositionModel] factorizer allocation
[DecompositionModel] decisions matrix allocation
[DecompositionModel] decomposition


In [4]:
_FROB_NORM = 1
_ITER = 150
estimator_params = DecompositionsEstimator.create_models_parameters(
    learning_rate=10**-4,
    generalization=10**-6,
)
estimator = DecompositionsEstimator(model=model, model_params=estimator_params)
for decision_type in DecisionType:
    for _ in range(_ITER):
        estimator.__fit__(
            target_decision=decision_type,
            n=_FROB_NORM,
        )
    # end : for (train_iters)
# end : for (decision_types)

[purchase] Adjust: 100%|██████████| 9935/9935 [00:01<00:00, 8643.21it/s]


In [5]:
recommender = ScoreBasedRecommender(
    estimator=estimator,
)
recommender.prediction()

Recommender.prediction(): 100%|██████████| 25662/25662 [00:30<00:00, 834.62it/s]


In [6]:
"""
recommender = ELABasedRecommender(
    estimator=estimator,
)
recommender.prediction()
"""

'\nrecommender = ELABasedRecommender(\n    estimator=estimator,\n)\nrecommender.prediction()\n'

In [7]:
evaluator = IRMetricsEvaluator(
    recommender=recommender,
    file_path=_TEST_SET_FILES_LIST[0],
)
evaluator.top_n_eval(
    top_n_conditions=top_n_conditions,
)
df: DataFrame = evaluator.evlautions_summary_df()
print(df)

[IO] /home/taegyu/git_repo/ipirec//data/colley/test_4_like_list.csv


[LOAD] test_4_like_list.csv: 100%|██████████| 5590/5590 [00:00<00:00, 73382.99it/s]


    Conditions  Precision    Recall  F1-score  Accuracy  Hits    TP     FP  \
0            3   0.332651  0.223624  0.267453  0.990889   975   975   1956   
1            5   0.261412  0.292890  0.276257  0.988586  1277  1277   3608   
2            7   0.221670  0.347706  0.270738  0.986068  1516  1516   5323   
3            9   0.191630  0.386468  0.256215  0.983311  1685  1685   7108   
4           11   0.170652  0.420642  0.242801  0.980486  1834  1834   8913   
5           13   0.154948  0.451376  0.230702  0.977610  1968  1968  10733   
6           15   0.141931  0.477064  0.218775  0.974659  2080  2080  12575   
7           17   0.131676  0.501606  0.208594  0.971691  2187  2187  14422   
8           19   0.122448  0.521330  0.198316  0.968651  2273  2273  16290   
9           21   0.115270  0.542431  0.190135  0.965631  2365  2365  18152   
10          23   0.108940  0.561468  0.182475  0.962581  2448  2448  20023   
11          25   0.103460  0.579587  0.175578  0.959517  2527  2

In [8]:
evaluator = IRMetricsEvaluator(
    recommender=recommender,
    file_path=_TEST_SET_FILES_LIST[1],
)
evaluator.top_n_eval(
    top_n_conditions=top_n_conditions,
)
df: DataFrame = evaluator.evlautions_summary_df()
print(df)

[IO] /home/taegyu/git_repo/ipirec//data/colley/test_4_purchase_list.csv


[LOAD] test_4_purchase_list.csv: 100%|██████████| 3087/3087 [00:00<00:00, 73315.12it/s]


    Conditions  Precision    Recall  F1-score  Accuracy  Hits   TP     FP  \
0            3   0.067802  0.135522  0.090384  0.993177   227  227   3121   
1            5   0.048925  0.162985  0.075258  0.989981   273  273   5307   
2            7   0.039299  0.183284  0.064720  0.986749   307  307   7505   
3            9   0.032457  0.194627  0.055636  0.983472   326  326   9718   
4           11   0.027696  0.202985  0.048742  0.980181   340  340  11936   
5           13   0.024469  0.211940  0.043873  0.976892   355  355  14153   
6           15   0.022162  0.221493  0.040293  0.973607   371  371  16369   
7           17   0.020451  0.231642  0.037584  0.970324   388  388  18584   
8           19   0.018864  0.238806  0.034967  0.967027   400  400  20804   
9           21   0.017665  0.247164  0.032974  0.963735   414  414  23022   
10          23   0.016791  0.257313  0.031525  0.960453   431  431  25237   
11          25   0.015699  0.261493  0.029620  0.957140   438  438  27462   

In [9]:
tags_scores_evaluator = TagsScoreRMSEEvaluator(
    recommender=recommender,
    file_path=_TEST_SET_FILES_LIST[0],
)

tags_scores_evaluator.eval()

[IO] /home/taegyu/git_repo/ipirec//data/colley/test_4_like_list.csv


[LOAD] test_4_like_list.csv: 100%|██████████| 5590/5590 [00:00<00:00, 80230.50it/s]

Hits RMSE: 0.6782621320590287
ForAll RMSE: 0.6727886852089244


In [10]:
tags_scores_evaluator = TagsScoreRMSEEvaluator(
    recommender=recommender,
    file_path=_TEST_SET_FILES_LIST[1],
)

tags_scores_evaluator.eval()

[IO] /home/taegyu/git_repo/ipirec//data/colley/test_4_purchase_list.csv


[LOAD] test_4_purchase_list.csv: 100%|██████████| 3087/3087 [00:00<00:00, 77012.58it/s]

Hits RMSE: 0.6208070692602989
ForAll RMSE: 0.8740220243978206


In [11]:
from rec_tags_freq import CosineItemsTagsFreqAddPenalty

rec_tags_freq_dist = CosineItemsTagsFreqAddPenalty()
evaluator = IRMetricsEvaluator(
    recommender=recommender,
    file_path=_TEST_SET_FILES_LIST[0],
)
avg_cos_dist = rec_tags_freq_dist.tags_freq_distance(
    test_set=evaluator.TEST_SET_LIST,
    recommender=recommender
)
print(f"L: {avg_cos_dist}")

evaluator = IRMetricsEvaluator(
    recommender=recommender,
    file_path=_TEST_SET_FILES_LIST[1],
)
avg_cos_dist = rec_tags_freq_dist.tags_freq_distance(
    test_set=evaluator.TEST_SET_LIST,
    recommender=recommender
)
print(f"P: {avg_cos_dist}")

[IO] /home/taegyu/git_repo/ipirec//data/colley/test_4_like_list.csv


[LOAD] test_4_like_list.csv: 100%|██████████| 5590/5590 [00:00<00:00, 78328.80it/s]


L: 0.0869991260843094
[IO] /home/taegyu/git_repo/ipirec//data/colley/test_4_purchase_list.csv


[LOAD] test_4_purchase_list.csv: 100%|██████████| 3087/3087 [00:00<00:00, 78604.53it/s]


P: 0.07883558156744756
